# My first python3 & SQL script

I am obscuring some part due to some confidential data & infrastructure I was working back then. I made this note solely for review and highlighting a new discoveries I've made with python since I never learnt SQL or telnet before.

The year was 2017, I was asked to log any data that is delivered through serial connection. The device is installed in remote area which connected by RS232-to-TCP device. 

The intended method to do this is installing the serial driver from the device and log the data from serial console. But it was such a headache to do so. So I did an alternative, connect the device's TCP port with telnet console.

In [ ]:
import telnetlib
tn = telnetlib.Telnet(host='192.168.x.x', port=2009)
line = str(tn.read_until(b'\n\n', 300))

With telnetlib, connect to the device address and port. I also get the specification that the data traffic always ended with newline character for every 5minutes. The data are received in byte object then converted to string as we can see below,

In [ ]:
print(line)

b'T#01 85.07,\r\nT#03 87.69,90.69,89.89,17.91,12.78\r\n \r\n '


I just only need to log every comma delimite number after 'T#03' and send them to database.

In [ ]:
array = line[line.index("T#03")+5:line.index("\\r\\n \\r\\n")].split(",")
print(array)

['87.69', '90.69', '89.89', '17.91', '12.78']


First connect to database with the credential. I provided connect() method with host and port address, username, password and database name. I wish I knew better connect method than passing password as text like that.

In [ ]:
import MySQLdb
db = MySQLdb.connect(host="localhost", port=3306, user="username", passwd="password", db="database")
cursor = db.cursor()

Now that we get the cursor object, we can prepare the syntax to send the data we gathered before. I made a timestamp since it was required for time series analytics.

In [ ]:
from datetime import datetime
time1 = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
sql = "INSERT INTO table_0 (dtime, temp1, temp2, temp3, temp4, V_bat) \
        VALUES ('%s', '%s', '%s', '%s', '%s', '%s')" \
        %(time1, array[0], array[1], array[2], array[3], array[4])

With that the cursor object will try to execute the syntax then commit the changes. If it failed, the database will be sent to the last state before the changes happened.

In [ ]:
try:
    cursor.execute(sql)
    db.commit()
except Exception as e:
    db.rollback()
db.close()

Last step is close the connection to database.